In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import healpy as hp
import kalepy as kale

import holodeck as holo
from holodeck import detstats, anisotropy, plot, utils
from holodeck.constants import YR, MSOL, GYR

# Sato-Polito Process fow SWGB Modeling

1) Predict number of binaries in each bin in redshift, mass, and frequency

2) Obtain number of binaries for each realization in each bin by Poisson sampling

3) Uniformly sample angular positions (in $\cos \theta$ and $\phi$)

4) Get strain amplitude of each using eq. (7)
$$ h^2(z, \mathcal{M}, f) = \frac{32 \pi^{4/3}}{5 c^8} \frac{(1+z)^{10/3}}{d^2_L (z)} (\mathcal{G} \mathcal{M})^{10/3} f^{4/3} $$

5) Plug strain amplitude into Eq. (17)

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{f}{4\pi \Delta f}   \int d \vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h^2 (f,\vec{\theta})   \bigg)^2 
+ \big( \frac{f}{4 \pi \Delta f}\big)^2 \int d\vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h^4 (f, \vec{\theta})
$$


## Applying to our methods


$\frac{d N_{\Delta f}}{d \vec{\theta}}$ is the number in that frequency bin, at that angle

$ h_c^2 = \frac{f}{df} h_s^2 $ so this is the same as
$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{1}{4\pi}   \int d \vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h_c^2 (f,\vec{\theta})   \bigg)^2 
+ \bigg( \frac{1}{4 \pi}\bigg)^2 \int d\vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h_c^4 (f, \vec{\theta} )
$$

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{1}{4\pi}   \int d \vec{\theta} h_c^2 (f)   \bigg)^2 
+ \bigg( \frac{1}{4 \pi} \bigg)^2 \int d\vec{\theta} h_c^4 (f )
$$

We already have a characteristic strain at each pixel, having previously done $\int \frac{dN_{\Delta f}}{dM dq dz} h_c^2(M,q,z) dM dq dz$ and distributing this total strain among pixels (using an evenly spread background and individual sources at individual pixels). Now, we just need to integrate that $h_c$ over all the pixels.

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{1}{4\pi}   \int d \vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h_c^2 (f,\vec{\theta})   \bigg)^2 
+ \bigg( \frac{1}{4 \pi}\bigg)^2 \int d\vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h_c^4 (f, \vec{\theta} )
$$

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{1}{4\pi}   \int d \vec{\theta} h_c^2 (f)   \bigg)^2 
+ \bigg( \frac{1}{4 \pi} \bigg)^2 \int d\vec{\theta} h_c^4 (f )
$$
where $\vec{\theta}$ is now just position bin, not also M,q,z

Need to convert $ d\vec{\theta}$ to $d (\mathrm{pixel})$


$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{1}{4\pi}   \sum  d\theta d\phi  h_c^2 (f)   \bigg)^2 
+ \bigg( \frac{1}{4 \pi} \bigg)^2 \sum d\theta d\phi h_c^4 (f )
$$

The 1/4pi, but this doesn't matter if I normalize C_l by C_0.

Pixel area is given in square degrees, dA = dtheta dphi. It is the same for all pixels, so can be pulled out of the integral/sum

$$ \frac{C_\ell (f)}{C_0 (f)} = \frac{\delta_{\ell 0}\delta_{m0} \bigg( \sum_\mathrm{pixels}   A_\mathrm{pix} h_c^2 (f)   \bigg)^2 
+ \sum_\mathrm{pixels}  A_\mathrm{pix} h_c^4 (f )}{\delta_{0 0}\delta_{00} \bigg( \sum_\mathrm{pixels}   A_\mathrm{pix} h_c^2 (f)   \bigg)^2 
+ \sum_\mathrm{pixels}  A_\mathrm{pix} h_c^4 (f )}
$$
simplifying for $l = l> 0$
$$ \frac{C_{\ell>0} (f)}{C_0 (f)} = \frac{\sum_\mathrm{pixels}  h_c^4 (f )}{ A_\mathrm{pix} \bigg( \sum_\mathrm{pixels}  h_c^2 (f)   \bigg)^2 
+ \sum_\mathrm{pixels}   h_c^4 (f )}
$$ 

$$ \frac{C_{\ell>0} (f)}{C_0 (f)} = \sum_\mathrm{pixels} \frac{ h_c^4 (f )}{ A_\mathrm{pix} \big(  h_c^2 (f)   \big)^2 
+  h_c^4 (f )}
$$ 


$$ \frac{C_{\ell>0} (f)}{C_0 (f)} = \sum_\mathrm{pixels} \frac{ h_c^4 (f )}{ A_\mathrm{pix} h_c^4 (f)  
+  h_c^4 (f )}
$$ 



# Set Up
### Read in Strain Data

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-16-mbp-ss19_uniform05A_n1000_r50_d20_f30_l2000_p0/'
hdfname = sspath+'ss_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][...]
hc_bg = ssfile['hc_bg'][...]
fobs = ssfile['fobs'][:]
dfobs = ssfile['dfobs'][:]
ssfile.close()

shape = hc_ss.shape
nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
print('N,F,R,L =', nsamps, nfreqs, nreals, nloudest)


### Get best sample

In [ ]:
hc_ref15_10yr = 11.2*10**-15 
nsort, fidx, hc_tt, hc_ref15 = detstats.rank_samples(hc_ss, hc_bg, fobs, hc_ref=hc_ref15_10yr, ret_all=True)
print(hc_ref15)

### Get healpix map

In [ ]:
nside=32
moll_hc = anisotropy.healpix_map(hc_ss[nsort[0]], hc_bg[nsort[0]], nside=nside)

In [ ]:
rr=0
hp.mollview(moll_hc[rr,fidx], title='Sample %d, Realization %d, $f$=%.2f yr$^{-1}$' % (nsort[0], rr, fobs[fidx]*YR))

# Calculate Anisotropy

$$ \frac{C_{\ell>0} (f)}{C_0 (f)} = \sum_\mathrm{pixels} \frac{ h_c^4 (f )}{ A_\mathrm{pix} h_c^4 (f)  
+  h_c^4 (f )}
$$ 

In [ ]:
print(moll_hc.shape)

In [ ]:
def ClC0_analytic(moll_hc):
    """ Calculate Cl/C0 for l>0 using Sato-Polito Eq. 17, modified
     to use characteristic strains from  Poisson sampled parameter bins 
     already calculated and placed at random pixels.

     Parameters
     ----------
     moll_hc : (F,R,npix) 
     """
    
    nside = hp.npix2nside(len(moll_hc[0,0]))
    area = hp.nside2pixarea(nside)

    sum_term = moll_hc**4 / (area*moll_hc**4 + moll_hc**4)
    ClC0 = np.sum(sum_term, axis=-1) # sum over pixels

    return ClC0

print(hp.nside2pixarea(nside))

In [ ]:
ClC0 = ClC0_analytic(moll_hc)
print(ClC0.shape) # F, R

In [ ]:
nshow=20

fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')
xx = fobs*YR
for rr in range(nshow):
    yy = ClC0[:,rr]
    ax.plot(xx, yy, color='tab:orange')
# ax.set_ylim(10**-6, 10**0)
plot._twin_hz(ax, nano=False)

# Back to dN/bin

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{f}{4\pi \Delta f}   \int d \vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h^2 (f,\vec{\theta})   \bigg)^2 
+ \big( \frac{f}{4 \pi \Delta f}\big)^2 \int d\vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h^4 (f, \vec{\theta})
$$


In [ ]:
dur, cad = 16.03*YR, 0.2*YR
fobs_gw_cents = utils.nyquist_freqs(dur,cad)
fobs_gw_edges = utils.nyquist_freqs_edges(dur,cad)
sam = holo.sam.Semi_Analytic_Model()
# sam = holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e11*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))  # faster version


In [ ]:
fobs_orb_cents = fobs_gw_cents/2.0
fobs_orb_edges = fobs_gw_edges/2.0
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)
redz_final, diff_num = holo.sam_cython.dynamic_binary_number_at_fobs(
    fobs_orb_cents, sam, hard, holo.cosmo)
edges = [sam.mtot, sam.mrat, sam.redz, fobs_orb_edges]
number = holo.sam_cython.integrate_differential_number_3dx1d(edges, diff_num)

In [ ]:
hs = holo.gravwaves.strain_amp_from_bin_edges_redz(edges, redz_final)
print(hs.shape) # (M,Q,Z,F)
print(number.shape) # (M,Q,Z,F)

In [ ]:
def Cl_analytic_from_num(fobs_orb_edges, number, hs, realize = False):
    """ Calculate Cl using Eq. (17) of Sato-Polito & Kamionkowski
    Parameters
    ----------
    fobs_orb_edges : (F,) 1Darray
        Observed orbital frequency bin edges
    hs : (M,Q,Z,F) NDarray
        Strain amplitude of each M,q,z bin
    number : (M,Q,Z,F) NDarray
        Number of sources in each M,q,z, bin
    
    """

    df = np.diff(fobs_orb_edges)                 #: frequency bin widths
    fc = kale.utils.midpoints(fobs_orb_edges)    #: use frequency-bin centers for strain (more accurate!)

    # df = fobs_orb_widths[np.newaxis, np.newaxis, np.newaxis, :] # (M,Q,Z,F) NDarray
    # fc = fobs_orb_cents[np.newaxis, np.newaxis, np.newaxis, :]  # (M,Q,Z,F) NDarray

    delta_term = (
        fc / (4*np.pi * df) * np.sum(number*hs**2, axis=(0,1,2))
    )**2

    Cl = (
        (fc / (4*np.pi*df))**2 * np.sum(number*hs**4, axis=(0,1,2))
    )

    C0 = Cl + delta_term

    return C0, Cl

C0, Cl = Cl_analytic_from_num(fobs_orb_edges, number, hs)

In [ ]:
nshow=20

fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_HZ, ylabel='$C_{\ell>0}/C_0$')
xx = fobs_gw_cents
yy = Cl/C0 # (F,)
ax.plot(xx, yy, color='tab:orange', label='Eq (17) with cython number, full sam')
# ax.set_ylim(10**-6, 10**0)
plot._twin_yr(ax, nano=False)
ax.legend()

In [ ]:
def Cl_analytic_from_num(fobs_orb_edges, number, hs, realize = False):
    """ Calculate Cl using Eq. (17) of Sato-Polito & Kamionkowski
    Parameters
    ----------
    fobs_orb_edges : (F,) 1Darray
        Observed orbital frequency bin edges
    hs : (M,Q,Z,F) NDarray
        Strain amplitude of each M,q,z bin
    number : (M,Q,Z,F) NDarray
        Number of sources in each M,q,z, bin
    realize : boolean or integer
        How many realizations to Poisson sample.
    
    """

    df = np.diff(fobs_orb_edges)                 #: frequency bin widths
    fc = kale.utils.midpoints(fobs_orb_edges)    #: frequency-bin centers 

    # df = fobs_orb_widths[np.newaxis, np.newaxis, np.newaxis, :] # (M,Q,Z,F) NDarray
    # fc = fobs_orb_cents[np.newaxis, np.newaxis, np.newaxis, :]  # (M,Q,Z,F) NDarray


    # Poisson sample number in each bin
    if utils.isinteger(realize):
        number = np.random.poisson(number[...,np.newaxis], 
                                size = (number.shape + (realize,)))
        df = df[...,np.newaxis]
        fc = fc[...,np.newaxis]
        hs = hs[...,np.newaxis]
    elif realize is True:
        number = holo.gravwaves.poisson_as_needed(number)



    delta_term = (fc/(4*np.pi*df) * np.sum(number*hs**2, axis=(0,1,2)))**2

    Cl = (fc/(4*np.pi*df))**2 * np.sum(number*hs**4, axis=(0,1,2))

    C0 = Cl + delta_term

    return C0, Cl

C0, Cl = Cl_analytic_from_num(fobs_orb_edges, number, hs, realize=False)
C0_many, Cl_many = Cl_analytic_from_num(fobs_orb_edges, number, hs, realize=20)

In [ ]:
nshow=20

fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_HZ, ylabel='$C_{\ell>0}/C_0$')
xx = fobs_gw_cents
yy = Cl/C0 # (F,)
ax.plot(xx, yy, color='tab:grey', label='Eq (17) with cython number, full sam')
rr = 0
ax.plot(xx, Cl_many[:,rr]/C0_many[:,rr], color='tab:orange', alpha=0.25, label = 'Poisson number/bin realization')
for rr in range(1, nshow):
    ax.plot(xx, Cl_many[:,rr]/C0_many[:,rr], color='tab:orange', alpha=0.25)
# ax.set_ylim(10**-6, 10**0)
plot._twin_yr(ax, nano=False)
ax.legend()

In [ ]:
nshow=20



def draw_analytic(ax, Cl, C0, fobs_gw_cents):
    xx = fobs_gw_cents
    yy = Cl/C0 # (F,)
    ax.plot(xx, yy, color='tab:orange', lw=2, label='Eq (17) with cython number, full sam')
    rr = 0
    ax.plot(xx, Cl_many[:,rr]/C0_many[:,rr], color='tab:orange', alpha=0.15, linestyle='-', 
            label = 'Poisson number/bin realization')
    for rr in range(1, nshow):
        ax.plot(xx, Cl_many[:,rr]/C0_many[:,rr], color='tab:orange', alpha=0.25, linestyle='-')

def draw_spk(ax):
    spk_xx= np.array([3.5*10**-9, 1.25*10**-8, 1*10**-7]) /YR
    spk_yy= np.array([1*10**-5, 1*10**-3, 1*10**-1])
    ax.plot(spk_xx * YR, spk_yy, label='SP & K Rough Estimate', color='limegreen', ls='--')

def draw_bayes(ax, lmax, colors = ['k', 'b', 'r', 'g', 'c', 'm']):
    xx_Nihan = np.array([2.0, 4.0, 5.9, 7.9, 9.9]) *10**-9 # Hz
    
    Cl_nihan = np.array([
    [0.20216773, 0.14690035, 0.09676646, 0.07453352, 0.05500382, 0.03177427],
    [0.21201336, 0.14884939, 0.10545698, 0.07734305, 0.05257189, 0.03090662],
    [0.20840993, 0.14836757, 0.09854803, 0.07205384, 0.05409881, 0.03305785],
    [0.19788951, 0.15765126, 0.09615489, 0.07475364, 0.0527356 , 0.03113331],
    [0.20182648, 0.14745265, 0.09681202, 0.0746824 , 0.05503161, 0.0317012 ]])
    for ll in range(lmax):
        ax.plot(xx_Nihan, Cl_nihan[:,ll]/Cl_nihan[:,0], 
                    label = '$l=%d$' % (ll+1), 
                color=colors[ll], marker='o', ms=8)

def plot_ClC0():
    fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_HZ, ylabel='$C_{\ell>0}/C_0$')
    draw_analytic(ax, Cl, C0, fobs_gw_cents)
    draw_spk(ax)
    draw_bayes(ax, lmax=6)
    # ax.set_ylim(10**-6, 10**0)
    plot._twin_yr(ax, nano=False)
    fig.legend(bbox_to_anchor=(0,-0.15), loc='upper left', bbox_transform = ax.transAxes, ncols=3)
    return fig

fig = plot_ClC0()

In [ ]:
# sph_harm_file = np.load('/Users/emigardiner/GWs/holodeck/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/anisotropy/sph_harm_lmax6_nside32_nbest100.npz')

# # load ss info
# shape = sph_harm_file['ss_shape']
# nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
# fobs = sph_harm_file['fobs']

# # load ranking info
# nsort = sph_harm_file['nsort']
# fidx = sph_harm_file['fidx']
# hc_tt = sph_harm_file['hc_tt']
# hc_ref15 = sph_harm_file['hc_ref15']

# # load harmonics info
# nside = sph_harm_file['nside']
# lmax  = sph_harm_file['lmax']
# moll_hc_best = sph_harm_file['moll_hc_best']
# Cl_best = sph_harm_file['Cl_best']
# nbest = len(moll_hc_best)

# sph_harm_file.close()

In [ ]:
def draw_ClC0_medians(ax, xx, Cl_best, lmax, nshow):

    yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
    yy = np.median(yy, axis=-1) # (B,F,l) median over realizations

    colors = ['k', 'b', 'r', 'g', 'c', 'm']
    for ll in range(lmax):
        ax.plot(xx, np.median(yy[:,:,ll], axis=0), color=colors[ll]) #, label='median of samples, $l=%d$' % ll)
        for pp in [50, 98]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,:,ll], percs, axis=0), alpha=0.1, color=colors[ll])
        
        for bb in range(0,nshow):
            # if ll==0 and bb==0:
            #     label = "individual best samples, median of realizations"
            # else: 
            label=None
            ax.plot(xx, yy[bb,:,ll], color=colors[ll], linestyle=':', alpha=0.1,
                                 linewidth=1, label=label)
def plot_ClC0():
    fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_HZ, ylabel='$C_{\ell>0}/C_0$')
    draw_analytic(ax, Cl, C0, fobs_gw_cents)
    draw_spk(ax)
    draw_bayes(ax, lmax=6)
    draw_ClC0_medians(ax, fobs_gw_cents, Cl_best, lmax, nshow=10)
    # ax.set_ylim(10**-6, 10**0)
    plot._twin_yr(ax, nano=False)
    ax.set_xlim(fobs[0]- 10**(-10), 1/YR)

    fig.legend(bbox_to_anchor=(0,-0.15), loc='upper left', bbox_transform = ax.transAxes, ncols=3)
    return fig

fig = plot_ClC0()         

# GPF

$$ P(M,q,z) = f_0' \bigg( \frac{M}{M_0}\bigg)^{\alpha_f}(1+z)^{\beta_f} $$

$$ P(M,q,z) = 0.03 \bigg( \frac{M}{1\times10^{11} M_\odot} \bigg)^{0} (1+z)^{0.8} $$

$$ P(M,q,z) = 0.03  (1+z)^{0.8} $$

In [ ]:
print('f_0=', sam._gpf._frac_norm)
print('m_alpha=', sam._gpf._malpha)
print('z_beta=', sam._gpf._zbeta)
print('q_gamma=', sam._gpf._qgamma)
print('m_ref (M_sun) = %e' % (sam._gpf._mref/MSOL))


In [ ]:
print(np.min(sam.redz), np.max(sam.redz))

In [ ]:
def P_of_z(z):
    return (0.03*(1+z)*0.8)

In [ ]:
P_of_z(0.001)
P_of_z(10)